# SI 618 WN 2018 - Lab 8: Clustering

## Submission Instructions:
Please turn in this Jupyter notebook file (both .ipynb and .html formats) on Canvas before midnight.

## Objectives:
- Be able to perform k-means clustering, bisecting k-means clustering
- Be able to execute various techniques for picking k
- Evaluate cluster quality

## Submission Instructions:
Please turn in via Canvas:
1. This Jupyter notebook file **in .html format** (not as an IPython or Jupyter notebook) 
2. The URL to the published version this notebook (you will need to publish it)

#### Load libraries

In [3]:
import pandas as pd
import numpy as np

In [4]:
ACCESS_KEY = 
SECRET_KEY = 
ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
AWS_BUCKET_NAME = "si618clustering"
MOUNT_NAME = "si618clustering"

# Uncomment the following line if you need to unmount the S3 bucket
# dbutils.fs.unmount("/mnt/si618image/")

# mount the S3 bucket
dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)

####Display the files in the S3 bucket

In [6]:
display(dbutils.fs.ls("/mnt/%s/" % MOUNT_NAME))

<h2 id="k-means">1. K-means</h2>

<p><a href="http://en.wikipedia.org/wiki/K-means_clustering">k-means</a> is one of the
most commonly used clustering algorithms that clusters the data points into a
predefined number of clusters. The MLlib implementation includes a parallelized
variant of the <a href="http://en.wikipedia.org/wiki/K-means%2B%2B">k-means++</a> method
called <a href="http://theory.stanford.edu/~sergei/papers/vldb12-kmpar.pdf">kmeans||</a>.</p>

<p><code>KMeans</code> is implemented as an <code>Estimator</code> and generates a <code>KMeansModel</code> as the base model.</p>

#### Estimator

1. An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.

2. For example, K-means, LDA, etc.



<h4 id="input-columns">Input Columns</h4>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>featuresCol</td>
      <td>Vector</td>
      <td>"features"</td>
      <td>Feature vector</td>
    </tr>
  </tbody>
</table>

<h4 id="output-columns">Output Columns</h4>

<table class="table">
  <thead>
    <tr>
      <th align="left">Param name</th>
      <th align="left">Type(s)</th>
      <th align="left">Default</th>
      <th align="left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>predictionCol</td>
      <td>Int</td>
      <td>"prediction"</td>
      <td>Predicted cluster center</td>
    </tr>
  </tbody>
</table>

Take a quick look at the code below which applies K-means algorithm on a sample dataset. We will ask you to adapt this code to a more fun dataset.
### Example code (just run this cell):

In [9]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Loads data.
dataset = spark.read.format("libsvm").load("/mnt/si618clustering/sample_kmeans_data.txt")

# Trains a k-means model with 2 clusters. Creates a K Means object, then use .fit to model it to the data.
kmeans = KMeans().setK(2).setSeed(1)
model = kmeans.fit(dataset)

# Make predictions
predictions = model.transform(dataset)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = model.computeCost(dataset)
print("Within Set Sum of Squared Errors = " + str(wssse))

# Evaluate clustering by computing Silhouette score (ranging between -1 and 1 -- 1 meaning the data point belongs to the cluster)
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

## Fun Activity: Clustering of 618 Class based on Music Tastes

Now let's try to create our own data and conduct K-means clustering step-by-step.

Enter your music tastes in this Google Sheet (scale: 1-10): https://docs.google.com/spreadsheets/d/18wIMBbQRGR8l65s6Tmsb6IJQWZ0VlFHdH3pmiGE3WLQ/edit?usp=sharing

### After everyone enters their music preferences, Kai will upload the dataset to AWS S3.
### Get started: Load the data we just created

In [12]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, StringType

schema = StructType([
    StructField("_c0", StringType()),
    StructField("Jazz", DoubleType()),
    StructField("Soul", DoubleType()),
    StructField("Pop", DoubleType()),
    StructField("R&B", DoubleType()),
    StructField("Opera", DoubleType()),
    StructField("Country", DoubleType()),
    StructField("Rock&Roll", DoubleType()),
])

music_raw = spark.read.csv("/mnt/si618clustering/music.tsv", inferSchema=True, header=True, sep='\t',schema=schema)

In [13]:
music_raw.show(30)

### Step 0 - Impute missing data

#### Spark MLLib cannot handle missing values (null values), so it's necessary to take care of them before training a model.

#### Transformer

1. A transformer is an abstraction which transforms a Spark DataFrame into another.

2. Prepares a Spark DataFrame for a Machine Learning algorithm.

3. For example, VectorAssembler, Imputer, etc.

### Example code (just run this cell):

In [18]:
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([
    (1.0, float("nan")),
    (2.0, float("nan")),
    (float("nan"), 3.0),
    (4.0, 4.0),
    (5.0, 5.0)
], ["a", "b"])

# The argument inputCols serves to tell Imputer which particular columns in our dataframe are to be imputed
# The argument outputCols serves to tell Imputer the names of the output columns
imputer = Imputer(inputCols=["a", "b"], outputCols=["out_a", "out_b"])
model = imputer.fit(df)
df = model.transform(df)
df.show()

### Model after the example to impute missing values in your music tastes features:

In [20]:
# The argument inputCols serves to tell Imputer which particular columns in our dataframe are to be imputed
# The argument outputCols serves to tell Imputer the names of the output columns
imputer = Imputer(inputCols=["Jazz", "Soul", "Pop", "R&B", "Opera", "Country", "Rock&Roll"], 
                  outputCols=["Jazz", "Soul", "Pop", "R&B", "Opera", "Country", "Rock&Roll"])
imputer_model = imputer.fit(music_raw) 
music_imputed = imputer_model.transform(music_raw) 
music_imputed.show(5)

###Step 1 - Assemble your features

In contrast to most ML packages out there, Spark ML requires your input features to be gathered in a single column of your dataframe, usually named features; and it provides a specific method for doing this, VectorAssembler:

####VectorAssembler

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees. 

VectorAssembler accepts the following input column types: all numeric types, boolean type, and vector type. In each row, the values of the input columns will be concatenated into a vector in the specified order.

### Example code (just run this cell):

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])

# The argument inputCols serves to tell VectoeAssembler which particular columns in our dataframe are to be used as features
assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show(truncate=False)

### Model after the example to assemble your music tastes features:

In [26]:
assembler = VectorAssembler(
    inputCols=["Jazz", "Soul", "Pop", "R&B", "Opera", "Country", "Rock&Roll"],
    outputCol="features") 

music_assembled = assembler.transform(music_imputed)
print("Assembled columns 'Jazz', 'Soul', 'Pop', 'R&B', 'Opera', 'Country', 'Rock&Roll' to vector column 'features'")
output.select("features").show(truncate=False)

### For Step 2 through Step 4, copy some of the code from the cell 10, and try to adapt it to our music dataset.

###Step 2 - Fit your K-means model

In [29]:
# Trains a k-means model with 3 clusters.
kmeans = KMeans().setK(3).setSeed(1)
kmeans_model = kmeans.fit(music_assembled) # music_assembled

###Step 3 - Predict using the model fit

In [31]:
# Make predictions
music_predictions = kmeans_model.transform(music_assembled)
music_predictions.show(5)

###Step 4 - Evalute your K-means model

#### Evaluating Clusters
Here is a quote from the scikit-learn's [clustering tutorial](http://scikit-learn.org/stable/modules/clustering.html#clustering-performance-evaluation) regarding clustering performance evaluation.

> Evaluating the performance of a clustering algorithm is not as trivial as counting the number of errors or the precision and recall of a supervised classification algorithm. In particular any evaluation metric should not take the absolute values of the cluster labels into account but rather if this clustering define separations of the data similar to some ground truth set of classes or satisfying some assumption such that members belong to the same class are more similar that members of different classes according to some similarity metric.

In [34]:
# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = kmeans_model.computeCost(music_assembled)
print("Within Set Sum of Squared Errors = " + str(wssse))

In [35]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(music_predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

###Step 4 - Show the cluster centers and cluster assignments

In [37]:
# Shows the clustering centers
centers = kmeans_model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

In [38]:
# Shows the clustering membership
music_transformed = kmeans_model.transform(music_assembled)
music_transformed.show(5)  

### Do you and your friends have similar music tastes? Do you belong to the same cluster? :)

YES. I'm so relieved. :)

### Step 5 - Visualize the results

#### Choose 3 dimensions("Jazz", "Soul", Pop") to visualize the clusters in a 3D plot

In [42]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D

pddf_pred = music_transformed.select("_c0","Pop","Country","Rock&Roll","prediction").toPandas().set_index('_c0')
pddf_pred.head()

threedee = plt.figure(figsize=(6,4)).gca(projection='3d')
threedee.scatter(pddf_pred.Pop, pddf_pred.Country, pddf_pred['Rock&Roll'], c=pddf_pred.prediction)
threedee.set_xlabel('Pop')
threedee.set_ylabel('Country')
threedee.set_zlabel('Rock&Roll')
display(threedee.figure)

#### Display a pairplot of the first three features, segmented by clusters.

In [44]:
display(kmeans_model, music_transformed.select("features"))

##2. Bisecting k-means

Bisecting k-means is a kind of hierarchical clustering using a divisive (or “top-down”) approach: all observations start in one cluster, and splits are performed recursively as one moves down the hierarchy.

Bisecting K-means can often be much faster than regular K-means, but it will generally produce a different clustering.

### Example code (just run this cell):

In [47]:
from pyspark.ml.clustering import BisectingKMeans

# Trains a bisecting k-means model.
bkm = BisectingKMeans().setK(2).setSeed(1)
model = bkm.fit(output)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
cost = model.computeCost(output)
print("Within Set Sum of Squared Errors = " + str(cost))

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = model.clusterCenters()
for center in centers:
    print(center)


### Model after the example to conduct a Bisecting K-means algorithm on the music tastes dataset with 3 clusters. Repeat Step 2 - Step 4 just like in the K-means clustering.

In [49]:
# Trains a bisecting k-means model.

bkm = BisectingKMeans().setK(3).setSeed(1)
bkm_model = bkm.fit(music_assembled)

music_predictions = bkm_model.transform(music_assembled)

# Evaluate clustering by computing Within Set Sum of Squared Errors.
wssse = bkm_model.computeCost(music_assembled)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(music_predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
print("Cluster Centers: ")
centers = bkm_model.clusterCenters()
for center in centers:
    print(center)

# Shows the clustering membership
music_transformed_bkm = bkm_model.transform(music_assembled)
music_transformed_bkm.show(5)  

## 3. Picking K - How many clusters?

A number of clustering methods, such as k-means, assumes the parameter _k_ (#clusters) is known in advance, which is often not the case in practice. A number of techniques exist for determining the number of clusters in a dataset. See [this Wikipedia page](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#Information_Criterion_Approach) for a detailed discussion.

In this section, we focus on four of the approaches:
0. Use your eyeballs
1. Rule of thumb
2. The Elbow Method
3. The Silhouette Approach

Let us see if all the methods listed above will be able to recover the true number of clusters.

## 3.1 Rule of thumb:
Choosing the number of clusters to simply be

$$
k \approx \sqrt{n/2}
$$

where n is the number of observations.

In [52]:
np.sqrt(music_assembled.count()/2)

## 3.2 The Elbow Method (Scree Plot)
Plot the percentage of variance explained as a function of the number of clusters.

You can reduce this error measure by increasing k. In fact the optimal k is usually one where there is an “elbow” in the elbow graph.

See [here](https://en.wikipedia.org/wiki/Determining_the_number_of_clusters_in_a_data_set#The_Elbow_Method) for an explanation.

In [54]:
cost = list()
for k in range(2,11):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    kmeans_model = kmeans.fit(music_assembled)
    cost.append(kmeans_model.computeCost(music_assembled)/k)

print(cost)

#### Plot the Within Set Sum of Squared Errors as a function of number of clusters

In [56]:
fig, ax = plt.subplots()
plt.plot(range(2,11), cost, 'b*-')
plt.xlabel('Number of clusters');
plt.ylabel('Within Set Sum of Squared Error');
plt.title('Elbow for K-Means clustering');

display(fig)

## 3.3 The Silhouette Approach
The silhouette coefficient can be used for evaluating clustering where there lacks ground truth. We can use the same metric for the purpose of finding the best k for the dataset.

In [58]:
cost = list()
evaluator = ClusteringEvaluator()
for k in range(2,11):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    kmeans_model = kmeans.fit(music_assembled)
    music_predictions = kmeans_model.transform(music_assembled)
    silhouette = evaluator.evaluate(music_predictions)
    cost.append(silhouette)

#### Plot the silhouette scores as a function of number of clusters

In [60]:
kIdx = np.argmax(cost)

fig, ax = plt.subplots()
plt.plot(range(2,11), cost, 'b*-')
plt.plot(range(2,11)[kIdx], cost[kIdx], marker='o', markersize=12, 
         markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.xlim(1, plt.xlim()[1])
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.title('Silhouette Scores for k-means clustering')

display(fig)

### Model after one of the above method to pick the best K for Bisecting K-means algorithm on our music dataset. What is the best K according to your analysis?

In [62]:
np.sqrt(music_assembled.count()/2)

We ran out of time!

# End of Lab 8